INTRODUCTION

# hiii  hey oui

# DATA PREPROCESSING
Setting the environment

In [25]:
import numpy as np
import os
from numpy.random import default_rng
import matplotlib.pyplot as plt
import pandas as pd

## Loading the DataSets

In [26]:
# Enable only the first time:
os.chdir("/Users/manav/Documents/defli's code/INDIAI/Datasets")

general_data = pd.read_csv('general_data.csv').copy()
employee_survey_data = pd.read_csv('employee_survey_data.csv').copy()
manager_survey_data = pd.read_csv('manager_survey_data.csv').copy()
in_time = pd.read_csv('in_time.csv').copy()
out_time = pd.read_csv('out_time.csv').copy()

The datasets will be processed separately, then once clean thet'll be merged into one.

## Dropping Unnecesary Attributes

### General Data

In [27]:
general_data

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,1.0,Y,11,8,0,1.0,6,1,0,0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,0.0,Y,23,8,1,6.0,3,5,1,4
2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,...,1.0,Y,15,8,3,5.0,2,5,0,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,...,3.0,Y,11,8,3,13.0,5,8,7,5
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,...,4.0,Y,12,8,2,9.0,2,6,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,42,No,Travel_Rarely,Research & Development,5,4,Medical,1,4406,Female,...,3.0,Y,17,8,1,10.0,5,3,0,2
4406,29,No,Travel_Rarely,Research & Development,2,4,Medical,1,4407,Male,...,2.0,Y,15,8,0,10.0,2,3,0,2
4407,25,No,Travel_Rarely,Research & Development,25,2,Life Sciences,1,4408,Male,...,0.0,Y,20,8,0,5.0,4,4,1,2
4408,42,No,Travel_Rarely,Sales,18,2,Medical,1,4409,Male,...,0.0,Y,14,8,1,10.0,2,9,7,8


## Treatment of Missing values

In [28]:
#os.chdir("./Datasets")

# Code to detect missing values
def DisplayMissingValues(data,data_name,display=False):
    
    #print("Missing values from:",data_name)
    df = pd.DataFrame(data)
    threshold = df.shape[0]

    filtered_df = df[df.columns[df.notnull().sum() < threshold]]
    
    if display:
        filtered_df.info()
        print()
    print("[",len(filtered_df.columns),"] attributes with missing values in:",data_name,"\n")


### General Data

In [29]:

DisplayMissingValues(general_data,"general_data",display=True)

total_employees = len(general_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   NumCompaniesWorked  4391 non-null   float64
 1   TotalWorkingYears   4401 non-null   float64
dtypes: float64(2)
memory usage: 69.0 KB

[ 2 ] attributes with missing values in: general_data 



2 attributes with missing values

### Employee Survey Data

In [30]:

DisplayMissingValues(employee_survey_data,"employee_survey_data",display=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EnvironmentSatisfaction  4385 non-null   float64
 1   JobSatisfaction          4390 non-null   float64
 2   WorkLifeBalance          4372 non-null   float64
dtypes: float64(3)
memory usage: 103.5 KB

[ 3 ] attributes with missing values in: employee_survey_data 



### Manager Survey Data

In [31]:

DisplayMissingValues(manager_survey_data,"manager_survey_data",display=False)

[ 0 ] attributes with missing values in: manager_survey_data 



### in_time and out_time

In [32]:

DisplayMissingValues(in_time,"in_time")


DisplayMissingValues(out_time,"out_time")

[ 261 ] attributes with missing values in: in_time 

[ 261 ] attributes with missing values in: out_time 



Too many attributes with missing values! We need a different approach for these two dataframes...

The following steps are to be followed:
1. Remove entrances/days with all N/A values for in or out time.
2. Merge both datasets into one by two key attributes: EmployeeID and Date.
3. Compute average hours of work per day per employee as a new attribute.
4. Compute deviation as a new attribute, to measure how dynamic the employee's routine is.

In [33]:
nonsence_entries_in = pd.DataFrame(in_time.loc[:, in_time.isnull().sum() == 4410])
print(nonsence_entries_in.shape[1]," days with all null values for time of entrance.")
nonsence_entries_in

12  days with all null values for time of entrance.


,2015-01-01,2015-01-14,2015-01-26,2015-03-05,2015-05-01,2015-07-17,2015-09-17,2015-10-02,2015-11-09,2015-11-10,2015-11-11,2015-12-25
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
nonsence_entries_out = pd.DataFrame(out_time.loc[:, in_time.isnull().sum() == 4410])
print(nonsence_entries_out.shape[1]," days with all null values for time of exit.")
nonsence_entries_out

12  days with all null values for time of exit.


,2015-01-01,2015-01-14,2015-01-26,2015-03-05,2015-05-01,2015-07-17,2015-09-17,2015-10-02,2015-11-09,2015-11-10,2015-11-11,2015-12-25
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
in_time = in_time.drop(columns=nonsence_entries_in.columns)
out_time = out_time.drop(columns=nonsence_entries_out.columns)

In [36]:
DisplayMissingValues(in_time,"in_time")
DisplayMissingValues(out_time,"out_time")

[ 249 ] attributes with missing values in: in_time 

[ 249 ] attributes with missing values in: out_time 



In [37]:
# verifying all columns in in_time exist in out_time and viceversa
# (homemade version)

in_t = in_time.columns
out_t = out_time.columns

for date_in_t in in_t:
    if date_in_t not in out_t:
        print(date_in_t)

for date_out_t in in_t:
    if date_out_t not in in_t:
        print(date_out_t)

By retiring the non-workable days, we reduced the columns with missing values by 12. But still, too many days with missing values.

In [38]:
def GetTime(datetime):
    if pd.notnull(datetime):
        return datetime[11:16]
    return None

def TimeToMinutes(time):
    if pd.notnull(time):
        return int(time[0:2])*60 + int(time[4:6])
    return 0

In [39]:
working_hours = pd.DataFrame(columns=["EmployeeID","AvgWorkingHours","RecentInasistances"])
for id in range (1,total_employees):
    # para calcular el promedio: procuro que todo este ordenado por id del mas bajo al mas alto
    # voy contando las inasistencias
    in_time[1:][id].sum()
    out_time[1:][id]
    avg_working_hours = round(round((TimeToMinutes(out_t)-TimeToMinutes(in_t))/60,2).sum()/emp["WORKED_HOURS"].ne(0).sum(),2)
    total_inasistances = emp["WORKED_HOURS"].eq(0).sum()
    working_hours.loc[len(working_hours)] = (id,avg_working_hours,total_inasistances)

KeyError: 1

In [ ]:
import pandas as pd
import numpy as np

# Placeholder for your datasets (replace with your actual loading code if different)
in_time = pd.read_csv('in_time.csv'); out_time = pd.read_csv('out_time.csv')
# For this example, I assume in_time and out_time are already loaded DataFrames

# Step 1: Function to parse time strings into minutes
def parse_time(s):
    if pd.isna(s) or s == '' or not isinstance(s, str):
        return np.nan
    try:
        h, m = s.split(':')
        return int(h) * 60 + int(m)
    except:
        return np.nan

# Step 2: Identify day columns (exclude the EmployeeID column)
day_columns = in_time.columns[1:]  # Assuming first column is 'Unnamed: 0' or similar

# Step 3: Convert all time entries to minutes
in_time_minutes = in_time[day_columns].applymap(parse_time)
out_time_minutes = out_time[day_columns].applymap(parse_time)

# Step 4: Calculate worked hours per day (in hours)
worked_hours = (out_time_minutes - in_time_minutes) / 60

# Step 5: Compute total worked hours per employee
total_worked_hours = worked_hours.sum(axis=1, skipna=True)

# Step 6: Count days where the employee actually worked (hours > 0 and not NaN)
worked_days = ((worked_hours > 0) & worked_hours.notna()).sum(axis=1)

# Step 7: Calculate average working hours per worked day
# If no days worked, set to 0 to avoid division by zero
avg_working_hours = np.where(worked_days > 0, total_worked_hours / worked_days, 0)

# Step 8: Calculate inassistances (days with no work or missing data)
inassistances = (worked_hours == 0) | worked_hours.isna()
total_inassistances = inassistances.sum(axis=1)

# Step 9: Create the result DataFrame
working_hours = pd.DataFrame({
    "EmployeeID": in_time['Unnamed: 0'],
    "AvgWorkingHours": np.round(avg_working_hours, 2),
    "RecentInasistances": total_inassistances
})

# Step 10: Display general statistics
total_workable_days = len(day_columns)  # Total days in the dataset
print("TOTAL WORKABLE DAYS:               ", total_workable_days)
print("AVERAGE WORKING HOURS PER WORK DAY:", round(working_hours['AvgWorkingHours'].mean(), 2))
print("AMOUNT OF RECENT INASSISTANCES:    ", working_hours['RecentInasistances'].sum())

# Step 11: Display results for all employees
print("\nWorking hours for all employees:")
print(working_hours.head(6))  # Show first 6 employees to match your output

/var/folders/vw/cl6_2pns2m5f8sp50m3z244m0000gn/T/ipykernel_20522/2007709416.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  in_time_minutes = in_time[day_columns].applymap(parse_time)
/var/folders/vw/cl6_2pns2m5f8sp50m3z244m0000gn/T/ipykernel_20522/2007709416.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  out_time_minutes = out_time[day_columns].applymap(parse_time)


TOTAL WORKABLE DAYS:                261
AVERAGE WORKING HOURS PER WORK DAY: 0.0
AMOUNT OF RECENT INASSISTANCES:     1151010

Working hours for all employees:
   EmployeeID  AvgWorkingHours  RecentInasistances
0           1              0.0                 261
1           2              0.0                 261
2           3              0.0                 261
3           4              0.0                 261
4           5              0.0                 261
5           6              0.0                 261


No missing values

In [41]:
emp = EmployeesWorkingHoursInfo_DF(in_time,out_time,4410)
avg_working_hours = round(emp["WORKED_HOURS"].sum()/emp["WORKED_HOURS"].ne(0).sum(),2)
total_inasistances = emp["WORKED_HOURS"].eq(0).sum()
working_hours.loc[len(working_hours)] = (4410,avg_working_hours,total_inasistances)

NameError: name 'EmployeesWorkingHoursInfo_DF' is not defined

In [44]:
copy_working_hours = working_hours.copy()
copy_working_hours["EmployeeID"] = copy_working_hours["EmployeeID"].astype(int)
copy_working_hours["RecentInasistances"] = copy_working_hours["RecentInasistances"].astype(int)
copy_working_hours

,EmployeeID,AvgWorkingHours,RecentInasistances


Merge all datasets into one

In [48]:
merge_key = "EmployeeID"
merged_df = pd.read_csv("general_data.csv")
merged_df = pd.merge(merged_df, pd.read_csv("employee_survey_data.csv"), on=merge_key, how="outer")
merged_df = pd.merge(merged_df, pd.read_csv("manager_survey_data.csv"), on=merge_key, how="outer")
merged_df = pd.merge(merged_df,copy_working_hours, on=merge_key, how="outer")

print(copy_working_hours.isnull().sum().sort_values(ascending=False))

columns_order = ["EmployeeID"] + [col for col in merged_df.columns if col != "EmployeeID"]
merged_df = merged_df[columns_order]
#sorted_merged_df = merged_df.sort_values(by="EmployeeID")
merged_df.to_csv("..\merged_data.csv", index=True)
merged_df





EmployeeID            0
AvgWorkingHours       0
RecentInasistances    0
dtype: int64


<>:12: SyntaxWarning: invalid escape sequence '\m'
<>:12: SyntaxWarning: invalid escape sequence '\m'
/var/folders/vw/cl6_2pns2m5f8sp50m3z244m0000gn/T/ipykernel_20522/2249660712.py:12: SyntaxWarning: invalid escape sequence '\m'
  merged_df.to_csv("..\merged_data.csv", index=True)


,EmployeeID,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,Gender,...,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,AvgWorkingHours,RecentInasistances
0,1,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,Female,...,1,0,0,3.0,4.0,2.0,3,3,NaN,NaN
1,2,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,Female,...,5,1,4,3.0,2.0,4.0,2,4,NaN,NaN
2,3,32,No,Travel_Frequently,Research & Development,17,4,Other,1,Male,...,5,0,3,2.0,2.0,1.0,3,3,NaN,NaN
3,4,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,Male,...,8,7,5,4.0,4.0,3.0,2,3,NaN,NaN
4,5,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,Male,...,6,0,4,4.0,1.0,3.0,3,3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,4406,42,No,Travel_Rarely,Research & Development,5,4,Medical,1,Female,...,3,0,2,4.0,1.0,3.0,3,3,NaN,NaN
4406,4407,29,No,Travel_Rarely,Research & Development,2,4,Medical,1,Male,...,3,0,2,4.0,4.0,3.0,2,3,NaN,NaN
4407,4408,25,No,Travel_Rarely,Research & Development,25,2,Life Sciences,1,Male,...,4,1,2,1.0,3.0,3.0,3,4,NaN,NaN
4408,4409,42,No,Travel_Rarely,Sales,18,2,Medical,1,Male,...,9,7,8,4.0,1.0,3.0,2,3,NaN,NaN
